### Known Issues (as of 06.12.19)

This is a python3 project, therefore the sys.version output below should show python 3 as used kernel
If thats not the case refer to 
``` bash
$ jupyter kernelspec list
```
which shows where the juypter kernels are defined at. Inside the shown folders lies a kernel.json which leads to the interpreter used by jupyter. For python3 it should show something along `"argv": [ "python3", "-m",...` for sure. For python2 accordingly. 

It's important to use `pip3` as installer, as we install for python3.

pyqt5 should be installed! Therefore:
``` bash
$ pip3 install qtpy5
```

Some further dependencies are given:
``` bash
$ pip3 install pandas scipy sympy colorama dill
```

In [1]:
import sys
print (sys.version)
print(sys.executable)
sys.path.append(r'../geo-adjust')

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
/usr/bin/python3


In [2]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (10,7)

from numpy import asarray, vstack, pi, nan, identity, mean, hstack, diag, array, int64, nanmean, concatenate, max, eye, hypot, insert
from scipy.spatial import KDTree
from scipy.signal import savgol_filter

from argparse import Namespace

from lasertracker import load_ros_log
from robot_arm import load_and_save_tf
from utils.misc import dcm2euler, euler2dcm, window_slicer

from utils.misc import format_figure
from utils.misc import sphere2cart

from geo_adjust.config import GHModelConfig
from geo_adjust.gh import GHAdjust



from IPython.display import display

# 1 Transformation Parameter and Mounting Vector

Schätzung der Stationierung und des Mounting Vektors

In [4]:
import pandas as pd
tf_f = r'data/tf_akig_080120.csv'
win_f = r'data/tslices_akig_080120.csv'
lt_f = r'data/lt_akig_080120.txt'

In [5]:
from lasertracker import load_ros_log

lt_df = load_ros_log(lt_f, version=2)
lt_df.dropna(inplace=True)
lt_df['stamp'] = pd.to_datetime(lt_df['t'], unit='s')
lt_df.head()

,t,sec,mus,hz,v,d,time,y,x,z,stamp
0,1.578493e+09,0,0,-0.708547,1.672367,6.884086,00:00:00,4.456602,5.200203,-0.698020,2020-01-08 14:20:43.274851561
1,1.578493e+09,0,10000,-0.708547,1.672368,6.884086,00:00:00.010000,4.456601,5.200202,-0.698026,2020-01-08 14:20:43.274990797
2,1.578493e+09,0,20000,-0.708546,1.672367,6.884086,00:00:00.020000,4.456597,5.200207,-0.698020,2020-01-08 14:20:43.275106907
3,1.578493e+09,0,30000,-0.708545,1.672368,6.884087,00:00:00.030000,4.456591,5.200212,-0.698027,2020-01-08 14:20:43.275152206
4,1.578493e+09,0,40000,-0.708545,1.672369,6.884087,00:00:00.040000,4.456591,5.200211,-0.698033,2020-01-08 14:20:43.275202513


In [6]:
lt_df['stamp'] = lt_df.loc[0, 'stamp'] + lt_df['time'] - pd.Timedelta(0.4, unit='s')
lt_df.set_index('stamp', drop=False, inplace=True)
lt_df.head()

,t,sec,mus,hz,v,d,time,y,x,z,stamp
stamp,,,,,,,,,,,
2020-01-08 14:20:42.874851561,1.578493e+09,0,0,-0.708547,1.672367,6.884086,00:00:00,4.456602,5.200203,-0.698020,2020-01-08 14:20:42.874851561
2020-01-08 14:20:42.884851561,1.578493e+09,0,10000,-0.708547,1.672368,6.884086,00:00:00.010000,4.456601,5.200202,-0.698026,2020-01-08 14:20:42.884851561
2020-01-08 14:20:42.894851561,1.578493e+09,0,20000,-0.708546,1.672367,6.884086,00:00:00.020000,4.456597,5.200207,-0.698020,2020-01-08 14:20:42.894851561
2020-01-08 14:20:42.904851561,1.578493e+09,0,30000,-0.708545,1.672368,6.884087,00:00:00.030000,4.456591,5.200212,-0.698027,2020-01-08 14:20:42.904851561
2020-01-08 14:20:42.914851561,1.578493e+09,0,40000,-0.708545,1.672369,6.884087,00:00:00.040000,4.456591,5.200211,-0.698033,2020-01-08 14:20:42.914851561


In [7]:
lt_df.shape

(10696, 11)

In [8]:
from utils.misc import window_grouper

lt_df = window_grouper(lt_df, win_f, margin=0.4)

In [9]:
plt.figure()
plt.plot(lt_df.x, '+')
pts = {}
for i in lt_df.group.unique(): 
    pts[i] = lt_df[lt_df.group==i].mean()
    plt.plot(lt_df.x[lt_df.group==i], '+')
format_figure()

/home/finn/.local/lib/python3.6/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [10]:
df_pts = pd.DataFrame.from_dict(pts, orient='index', columns=lt_df.mean().keys().tolist())
df_pts.sort_index(inplace=True)
df_pts.dropna(inplace=True)
df_pts.head()

,t,sec,mus,hz,v,d,time,y,x,z
J001,1.578493e+09,7.796380,473619.909502,-0.648032,1.508238,7.416607,00:00:08.270000,4.468042,5.901497,0.463666
J002,1.578493e+09,16.500000,495000.000000,-0.620986,1.618190,6.599020,00:00:16.995000,3.835245,5.360990,-0.312634
J003,1.578493e+09,23.063636,501363.636364,-0.621118,1.634263,6.540975,00:00:23.565000,3.798820,5.308598,-0.414856
J004,1.578493e+09,30.814480,475520.361991,-0.620835,1.609009,6.702970,00:00:31.290000,3.896369,5.448168,-0.256076
J005,1.578493e+09,38.153153,511846.846847,-0.602785,1.625662,6.601536,00:00:38.665000,3.737034,5.429898,-0.362015


In [11]:
rob_df = load_and_save_tf(tf_f)

In [12]:
df_pts_ur = window_slicer(rob_df, win_f, margin=0.4)

In [13]:
df_pts_ur.head()

,time,x,y,z,qx,qy,qz,qw,ut,k,...,dcm_2_var,dcm_3_var,dcm_4_var,dcm_5_var,dcm_6_var,dcm_7_var,dcm_8_var,ro_var,pi_var,ya_var
J001,1.578493e+09,-0.110486,0.095346,0.989254,-0.000030,-0.000271,-0.708158,0.706053,1.578493e+09,2769.5,...,3.732093e-09,6.455386e-15,6.999872e-10,8.450809e-10,8.450289e-10,3.731701e-09,6.797927e-16,3.731702e-09,8.450290e-10,6.995975e-10
J002,1.578493e+09,-0.910527,-0.112129,0.208389,0.001271,0.000906,-0.706239,0.707972,1.578493e+09,3860.0,...,9.488045e-10,7.704062e-15,7.717948e-10,3.112465e-10,3.111472e-10,9.491566e-10,3.255829e-15,9.491659e-10,3.111502e-10,7.720743e-10
J003,1.578493e+09,-0.939087,-0.111923,0.149852,0.184199,-0.182035,-0.682496,0.683471,1.578493e+09,4681.5,...,7.675249e-10,5.652424e-15,5.567940e-10,4.230721e-10,2.519210e-10,7.682593e-10,2.563923e-10,1.024657e-09,2.519227e-10,7.287132e-10
J004,1.578493e+09,-0.856540,-0.112080,0.244720,-0.181849,0.183906,-0.681942,0.684152,1.578493e+09,5647.0,...,7.601746e-10,7.938115e-15,6.205659e-10,3.800463e-10,3.013942e-10,7.605338e-10,2.530556e-10,1.013599e-09,3.013981e-10,7.000120e-10
J005,1.578493e+09,-0.910338,-0.170471,0.184146,0.271233,0.271836,-0.652129,0.653655,1.578493e+09,6569.0,...,7.208356e-10,1.494008e-10,5.751078e-10,1.475804e-10,1.475599e-10,1.257125e-09,1.492658e-10,2.528769e-09,2.968218e-10,2.235604e-09


In [14]:
#df_pts_ur = pd.read_pickle(tf_f.replace('.csv', '.pk'))
plt.figure()
plt.subplot(121)
plt.plot(df_pts_ur.x, df_pts_ur.y, '+')
plt.axis('equal')
plt.title('UR')
plt.subplot(122)
plt.plot(df_pts.x, df_pts.y, '+')
plt.axis('equal')
plt.title('LT')
format_figure()

In [15]:
params = {
    'atr_sigma': 20e-6,
    'var_rob': 0.00030 ** 2,  # robot position accuracy
    'var_rob_angle': (0.011 / 200 * pi) ** 2,  # robot pose angle accuracy
    'scale': True
}
p = Namespace(**params)

In [16]:
if p.scale:
    gh_config = GHModelConfig.load(r'data/models/lt_trafo_ccr_scale.pk')
else:
    gh_config = GHModelConfig.load(r'data/models/lt_trafo_ccr.pk')

solver = GHAdjust(gh_config, max_iterations=30)

# Initial parameters
if p.scale:
    x0 = [0., 0., -50. / 200 * pi, 0., -5.25, -6.25, -0.6, 0., 0., 0.]
else:
    x0 = [0., 0., -50. / 200 * pi, -5.25, -6.25, -0.6, 0., 0., 0.]
    
solver.set_initial_params(x0)

In [17]:
gh_config.get_model_syms()

(Matrix([
 [-e + ty + (m + 1)*(sin(pru)*sin(rru)*cos(yru) - sin(yru)*cos(rru))*(mv_x*sin(yeu)*cos(peu) + mv_y*(sin(peu)*sin(reu)*sin(yeu) + cos(reu)*cos(yeu)) + mv_z*(sin(peu)*sin(yeu)*cos(reu) - sin(reu)*cos(yeu)) + rob_y) + (m + 1)*(sin(pru)*cos(rru)*cos(yru) + sin(rru)*sin(yru))*(-mv_x*sin(peu) + mv_y*sin(reu)*cos(peu) + mv_z*cos(peu)*cos(reu) + rob_z) + (m + 1)*(mv_x*cos(peu)*cos(yeu) + mv_y*(sin(peu)*sin(reu)*cos(yeu) - sin(yeu)*cos(reu)) + mv_z*(sin(peu)*cos(reu)*cos(yeu) + sin(reu)*sin(yeu)) + rob_x)*cos(pru)*cos(yru)],
 [-n + tx + (m + 1)*(sin(pru)*sin(rru)*sin(yru) + cos(rru)*cos(yru))*(mv_x*sin(yeu)*cos(peu) + mv_y*(sin(peu)*sin(reu)*sin(yeu) + cos(reu)*cos(yeu)) + mv_z*(sin(peu)*sin(yeu)*cos(reu) - sin(reu)*cos(yeu)) + rob_y) + (m + 1)*(sin(pru)*sin(yru)*cos(rru) - sin(rru)*cos(yru))*(-mv_x*sin(peu) + mv_y*sin(reu)*cos(peu) + mv_z*cos(peu)*cos(reu) + rob_z) + (m + 1)*(mv_x*cos(peu)*cos(yeu) + mv_y*(sin(peu)*sin(reu)*cos(yeu) - sin(yeu)*cos(reu)) + mv_z*(sin(peu)*cos(reu)*cos

In [18]:
# remove outliers
for i in ['J006']:
    df_pts.drop(i, inplace=True, errors='ignore')
    df_pts_ur.drop(i, inplace=True, errors='ignore')

obs = hstack((df_pts_ur[['x', 'y', 'z', 'ro', 'pi', 'ya']].values, df_pts[['x', 'y', 'z']].values))
obs.shape[0]

12

In [19]:
sigma_ll = diag([p.var_rob] * obs.size)

for i in range(df_pts.shape[0]):
    sigma_ll[3 + i * 9:6 + i * 9, 3+i * 9:6 + i * 9] = diag([p.var_rob_angle] * 3)
    sigma_ll[6 + i * 9:9 + i * 9, 6 + i * 9:9 + i * 9] = diag([p.atr_sigma ** 2] * 3)

solver.add_data(obs, sigma_ll, var0=nanmean(diag(sigma_ll)),
                row_names=df_pts.index.tolist())

In [20]:
result = solver.solve()

In [21]:
x_sym, l_sym = gh_config.get_syms()
x_sym

[rru, pru, yru, m, ty, tx, tz, mv_x, mv_y, mv_z]

In [22]:
if p.scale:
    result.parameters.set_params_config(tuple(x_sym),
                                        ['gon'] * 3 + ['ppm'] + ['m'] * 3 + ['mm'] * 3,
                                        ['r', 'p', 'y', 'm', 'te', 'tn', 'th', 'mvx', 'mvy', 'mvz'],
                                        cxs=[lambda x: x / pi * 200] * 3 + [lambda x: x * 1e6] + [lambda x: x] * 3 + [lambda x: x * 1e3] * 3,
                                        formats=['.5f'] * 3 + ['.1f'] + ['.5f'] * 3 + ['.2f'] * 3)
else:
    result.parameters.set_params_config(tuple(x_sym),
                                        ['gon'] * 3 + ['m'] * 3 + ['mm'] * 3,
                                        ['r', 'p', 'y', 'te', 'tn', 'th', 'mvx', 'mvy', 'mvz'],
                                        cxs=[lambda x: x / pi * 200] * 3 + [lambda x: x] * 3 + [
                                            lambda x: x * 1e3] * 3,
                                        formats=['.5f'] * 6 + ['.2f'] * 3)

result.observations.set_obs_config(tuple(l_sym),
                                   ['mm'] * 3 + ['gon'] * 3 + ['mm'] * 3,
                                   ['lt_x', 'lt_y', 'lt_z', 'ro', 'pi', 'ya', 'rob_x', 'rob_y', 'rob_z'],
                                   cxs=[lambda x: x * 1e3] * 3 + [lambda x: x / pi * 200] * 3 + [
                                       lambda x: x * 1e3] * 3,
                                   formats=['.3f'] * 3 + ['.4f'] * 3 + ['.2f'] * 3,
                                   groups=['Robot Position']*3 + ['Robot Angle']*3 + ['Laser Tracker Position']*3)

In [23]:
result.model.print_summary()
result.parameters.print(print_correlation=False)
result.parameters.plot_corr()


 Result Summary 
 - Convergence: 				  [ OK]
    ├ # Iterations: 			    12
    └ Criterium: 					  1.000e-12
 - Hauptprobe: 				  [ OK]
    └ Max Eps: 					  -9.992e-16
 - Globaltest: 				  [NOK]
    ├ Test: 						  eq 
    ├ Var Prio: 					  0.000
    ├ Var Post: 					  0.000
    ├ Ratio:    					 658.230
    ├ Alpha: 						  0.100
    └ Testquantity: 				  0.592 < 658.230 < 1.496
 - Residuals:
    ├ SSE (vTv):                0.002
    ├ MSE:                      0.000
    ├ RMSE:                     0.004
    ├ WSSE (vTPv):              0.001
    ├ WMSE:                     0.000
    └ WRMSE:                    0.000



 Paramters
 - Parameter Estimates ± σ:
    ├  rru:     -0.18458 ±  0.50256   [gon]
    ├  pru:      0.63228 ±  0.42819   [gon]
    ├  yru:     38.48977 ±  0.45792   [gon]
    ├    m:       3262.6 ±   5508.7   [ppm]
    ├   ty:      6.08805 ±  0.00778   [m]
    ├   tx:      4.47446 ±  0.00837   [m]
    ├   tz:     -0.61544 ±  0.00639   [m]
    ├  mvₓ:         0.10 ±  

In [ ]:
obs = result.observations.dataframe()

In [ ]:
obs['Robot Position']['nv_max []'] = obs['Robot Position'][['nv_robₓ []', 'nv_rob_y []', 'nv_rob_z []']].max(axis=1)

In [ ]:
obs['Robot Position'].sort_values('nv_max []', ascending=False).head()

In [ ]:
obs['Robot Position'].sort_values('nv_rob_y []', ascending=False).head()
# obs['Robot Angle'].sort_values('nv_peu []', ascending=False).head()
# obs['Laser Tracker Position'].sort_values('nv_u []', ascending=False).head()